In [46]:
import numpy as np

from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env

In [47]:
class QRPfRA_v3(MujocoEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA/IK_Models/qrpfra_v3_leg_ik_scene_right.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = self.data.qpos.size + self.data.qvel.size

        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

    def step(self, action):
        x_position_before = self.data.qpos[0]

        self.do_simulation(action, self.frame_skip)
        x_position_after = self.data.qpos[0]

        observation = self._get_obs()
        reward = x_position_after - x_position_before
        info = {}

        if self.render_mode == "human":
            self.render()

        return observation, reward, False, False, info

    def _get_obs(self):
        #position = self.data.qpos.flat.copy()
        #velocity = self.data.qvel.flat.copy()
        sensor_data = self.data.sensordata.flat.copy()

        return sensor_data

    def reset_model(self):
        qpos = self.init_qpos
        qvel = self.init_qvel
        self.set_state(qpos, qvel)

        observation = self._get_obs()

        return observation

    def _get_reset_info(self):
        return {"works": True}



In [48]:
env = QRPfRA_v3()
check_env(env)
obs = env.reset()
print(obs)


(array([ 0.045 ,  0.1335, -0.107 ,  0.    ,  0.    ,  0.    ]), {'works': True})


In [49]:
left_leg_dataset = [] # [obs0, obs1, obs2, act0, act1, act2]

In [50]:
for i in range(-100, 100):
    for j in range(-100, 100):
        for k in range(-100, 100):
            #env.render_mode = "human"
            env_obs = []
            for step_cnt in range(0,12):
                obs, reward, done, _, info = env.step([float(m) for m in [i, j, k]])
                if step_cnt == 11:
                    env_obs = obs

            left_leg_dataset.append([env_obs[0], env_obs[1], env_obs[2], i/100, j/100, k/100])


env.close()

In [51]:
print(len(left_leg_dataset))

8000000


In [52]:
#Convert left leg list to numpy array
left_leg_dataset_as_np = np.array(left_leg_dataset)

In [53]:
print(left_leg_dataset_as_np[0:4])

observations = left_leg_dataset_as_np[:, 0:3][:]
print(observations)
actions = left_leg_dataset_as_np[:, 3:][:]
print(actions)

[[ 0.17738146 -0.14628534 -0.03948343 -1.         -1.         -1.        ]
 [ 0.17787466 -0.1452782  -0.04005906 -1.         -1.         -0.99      ]
 [ 0.17841268 -0.1441787  -0.04036878 -1.         -1.         -0.98      ]
 [ 0.1789408  -0.14307237 -0.0406731  -1.         -1.         -0.97      ]]
[[ 0.17738146 -0.14628534 -0.03948343]
 [ 0.17787466 -0.1452782  -0.04005906]
 [ 0.17841268 -0.1441787  -0.04036878]
 ...
 [ 0.06802939  0.04272428 -0.00187604]
 [ 0.06750877  0.04161429 -0.00218015]
 [ 0.06697809  0.04051071 -0.00249014]]
[[-1.   -1.   -1.  ]
 [-1.   -1.   -0.99]
 [-1.   -1.   -0.98]
 ...
 [ 0.99  0.99  0.97]
 [ 0.99  0.99  0.98]
 [ 0.99  0.99  0.99]]


In [54]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_obs, test_obs, train_actions, test_actions = train_test_split(observations, actions, test_size=0.1, shuffle=True)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='tanh')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the model
model.fit(train_obs, train_actions, epochs=10, batch_size=1024)

# Evaluate the model on the testing set
test_loss = model.evaluate(test_obs, test_actions)
print('Test Loss:', test_loss)


Epoch 1/10
7032/7032 [==============================] - 12s 2ms/step - loss: 0.0247 - accuracy: 0.9267
Epoch 2/10
7032/7032 [==============================] - 11s 2ms/step - loss: 0.0152 - accuracy: 0.9494
Epoch 3/10
7032/7032 [==============================] - 12s 2ms/step - loss: 0.0147 - accuracy: 0.9510
Epoch 4/10
7032/7032 [==============================] - 12s 2ms/step - loss: 0.0142 - accuracy: 0.9519
Epoch 5/10
7032/7032 [==============================] - 11s 2ms/step - loss: 0.0139 - accuracy: 0.9523
Epoch 6/10
7032/7032 [==============================] - 11s 2ms/step - loss: 0.0136 - accuracy: 0.9532
Epoch 7/10
7032/7032 [==============================] - 12s 2ms/step - loss: 0.0134 - accuracy: 0.9548
Epoch 8/10
7032/7032 [==============================] - 11s 2ms/step - loss: 0.0133 - accuracy: 0.9558
Epoch 9/10
7032/7032 [==============================] - 11s 2ms/step - loss: 0.0132 - accuracy: 0.9570
Epoch 10/10
25000/25000 [==============================] - 10s 391us/step

In [55]:
model.save("right_legs_model", overwrite=True)

INFO:tensorflow:Assets written to: right_legs_model/assets


INFO:tensorflow:Assets written to: right_legs_model/assets
